In [ ]:
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
import numpy as np
from scipy import sparse
from tqdm.notebook import tqdm

In [ ]:
from p_tqdm import p_map, p_umap

In [ ]:
!ls ../data/processed

In [ ]:
A_tr = sparse.load_npz('../data/processed/A_tr.npz')
B_tr = sparse.load_npz('../data/processed/B_tr.npz')
P_tr = sparse.load_npz('../data/processed/P_tr.npz')
A_tr_csr = A_tr
A_tr_csc = A_tr.tocsc(copy=True)  # memory is cheap ;D

In [ ]:
A_tr

## Sample paths strictly using node2vec

In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import sys
sys.path.insert(0, '../')

In [ ]:
from src.features import n2v
# del n2v
n2v = n2v.Node2Vec(A_tr, B_tr, P_tr)

In [ ]:
n2v.perform_one_walk_full()

In [ ]:
n2v.perform_one_walk_metapath()

In [ ]:
walks = n2v.save_corpus()

In [ ]:
%load_ext line_profiler

In [ ]:
!wc -l node2vec_n=1_p=2_q=1_wl=100.cor

In [ ]:
from gensim import utils

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = 'node2vec_n=1_p=2_q=1_wl=100.cor'
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield line.strip().split(' ')

In [ ]:
sentences = MyCorpus()

In [ ]:
%%time

import gensim.models

sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences, min_count=1, size=200)

In [ ]:
!ls HinDroid-with-Embeddings/data/processed/

In [ ]:
meta_tr = pd.read_csv('HinDroid-with-Embeddings/data/processed/meta_tr.csv', index_col=0)

In [ ]:
meta_tr.head()

In [ ]:
y_train = meta_tr.label == 'class1'

In [ ]:
app_vec = np.array([model.wv[f'app_{i}'] for i in range(len(meta_tr))])

In [ ]:
app_vec

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')

In [ ]:
svm.fit(app_vec, y_train)

In [ ]:
svm.score(app_vec, y_train)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        if 'api' in word: continue
        tokens.append(model.wv[word])
        labels.append(word)
    
    tsne_model = TSNE(n_components=2)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 12)) 
    for i in range(len(x)):
        color = 'b' if meta_tr.label.iloc[i] == 'class1' else 'r'
        plt.scatter(x[i],y[i],c=color)
#         plt.annotate(labels[i],
#                      xy=(x[i], y[i]),
#                      xytext=(5, 2),
#                      textcoords='offset points',
#                      ha='right',
#                      va='bottom')
    plt.show()

In [ ]:
tsne_plot(model)